# Notebook for the creation of the grid lsh hashes for both datasets

Sheet that converts the extracted data from the data/chosen_data folder to hashes that will be stored in data/hashed_data/disk

In [15]:
# Importing nescessary modules
import os
import sys
import shutil
from multiprocessing import Pool


def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Example usage
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

from utils.helpers.save_trajectory import save_trajectory_hashes
from utils.helpers import file_handler as fh
from utils.helpers import metafile_handler as mfh
from schemes.lsh_grid import GridLSH
import json

Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave


In [2]:
from constants import (
    P_MAX_LON,
    P_MIN_LON,
    P_MAX_LAT,
    P_MIN_LAT,
    R_MAX_LON,
    R_MIN_LON,
    R_MAX_LAT,
    R_MIN_LAT,
)

# Declaring global variables:

SHOULD_DELETE_OLD_FILES = True

PORTO_OUTPUT_FOLDER = "../../dataset/hashed_data/grid/porto/"
ROME_OUTPUT_FOLDER = "../../dataset/hashed_data/grid/rome/"

PORTO_DATA_FOLDER = "../../dataset/porto/output/"
ROME_DATA_FOLDER = "../../dataset/rome/output/"


# Rome LSH Grid


In [3]:
# Run this cell to clear the chosen files in the ROME folder

if SHOULD_DELETE_OLD_FILES:
    fh.delete_old_files(ROME_OUTPUT_FOLDER, ".gitkeep")

In [4]:
# Create Grid hash object for Rome and saves them to output folder. Also copies the metafiles denoting the different datasets

NUMBER_OF_TRAJECTORIES = 3050

resolution = 0.6  # km
layers = 4
meta_file = f"{ROME_OUTPUT_FOLDER}META-{NUMBER_OF_TRAJECTORIES}.txt"

GridRome = GridLSH(
    "Rome G1",
    R_MIN_LAT,
    R_MAX_LAT,
    R_MIN_LON,
    R_MAX_LON,
    resolution,
    layers,
    meta_file,
    ROME_DATA_FOLDER,
)

In [5]:
# Copying the meta_files:
meta_files = mfh.get_meta_files(ROME_DATA_FOLDER)

for filename in meta_files:
    shutil.copy(ROME_DATA_FOLDER + filename, ROME_OUTPUT_FOLDER)

# Generate the hashes and save them to output folder

hashes = GridRome.compute_dataset_hashes()

save_trajectory_hashes(ROME_OUTPUT_FOLDER, hashes)

data = {
    "city": "rome",
    "resolution": resolution,
    "layers": layers,
    "number_of_trajectories": NUMBER_OF_TRAJECTORIES,
}

with open('rome_lsh_grid_current_parameters.json', 'w') as file:
    json.dump(data, file)  


# Porto LSH Grid


In [7]:
# Run this cell to clear the chosen files in the PORTO folder

if SHOULD_DELETE_OLD_FILES:
    fh.delete_old_files(PORTO_OUTPUT_FOLDER, ".gitkeep")

In [9]:
# Create Grid LSH objec for Porto

NUMBER_OF_TRAJECTORIES = 3000


resolution = 0.6  # km
layers = 4
meta_file = f"{PORTO_OUTPUT_FOLDER}META-{NUMBER_OF_TRAJECTORIES}.txt"

GridPorto = GridLSH(
    "Porto G1",
    P_MIN_LAT,
    P_MAX_LAT,
    P_MIN_LON,
    P_MAX_LON,
    resolution,
    layers,
    meta_file,
    PORTO_DATA_FOLDER,
)

In [10]:
# Copying the meta_files
meta_files = mfh.get_meta_files(PORTO_DATA_FOLDER)

for filename in meta_files:
    shutil.copy(PORTO_DATA_FOLDER + filename, PORTO_OUTPUT_FOLDER)
    
# Creating the hashes and saving them to output folder

hashes = GridPorto.compute_dataset_hashes()

save_trajectory_hashes(PORTO_OUTPUT_FOLDER, hashes)

data = {
    "city": "porto",
    "resolution": resolution,
    "layers": layers,
    "number_of_trajectories": NUMBER_OF_TRAJECTORIES,
}

with open('porto_lsh_grid_current_parameters.json', 'w') as file:
    json.dump(data, file)  



# Visualize original and hashed trajectory

In [16]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from schemes.lsh_grid import GridLSH
from constants import  *
from utils.helpers import file_handler
import pprint

In [17]:
CITY = "rome"

In [18]:
if CITY == "rome":
    TRUE_DATA_FOLDER = "../../dataset/rome/output/"
    PREFIX = "R"
    GRID = GridRome
    HASH_DATA_FOLDER = "../../dataset/hashed_data/grid/rome/"

if CITY == "porto":
    TRUE_DATA_FOLDER = "../dataset/porto/output/"
    PREFIX = "P"
    GRID = GridPorto
    HASH_DATA_FOLDER = "../dataset/hashed_data/grid/porto/"

In [19]:
all_true_trajectories = file_handler.load_all_trajectory_files(TRUE_DATA_FOLDER, PREFIX)
true_trajectory = all_true_trajectories["R_ABU"]

all_hashed_trajectories = file_handler.load_all_trajectory_hashes(HASH_DATA_FOLDER, PREFIX)
hashed_trajectory = all_hashed_trajectories["R_ABU"]

In [20]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_trajectory(grid_lsh, trajectory, hashed_trajectory):
    """
    Visualizes the grid of the GridLSH object using Folium and adds a single trajectory.

    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    - trajectory (list): List of (lat, lon) coordinates representing a single trajectory.
    - hashed_trajectory (list of lists): Hashed trajectory representation per layer.

    Returns:
    - A Folium map object.
    """

    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different grid layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each grid layer
        grid_layer = FeatureGroupSubGroup(base_layer, name=f"Grid Layer {layer_index + 1}")
        map_grid.add_child(grid_layer)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(grid_layer)

    # Plot the original trajectory as a blue polyline
    folium.PolyLine(
        trajectory,
        color="blue",
        weight=2.5,
        opacity=1,
        popup="Trajectory Path"
    ).add_to(base_layer)

    # Add markers for each point along the original trajectory
    for lat, lon in trajectory:
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,  # Small marker
            color="black",
            fill=True,
            fill_opacity=1,
            popup=f"Point: ({lat:.5f}, {lon:.5f})"
        ).add_to(base_layer)
        
    # Plot hashed trajectory points as markers with circles (in the same FeatureGroupSubGroup)
    hashed_layer_colors = ["red", "blue", "green", "purple", "orange"]  # Colors for different layers

    for layer_index, layer_points in enumerate(hashed_trajectory):
        layer_color = hashed_layer_colors[layer_index % len(hashed_layer_colors)]  # Assign color per layer

        # Create a single FeatureGroupSubGroup for both circles and markers
        hashed_layer = FeatureGroupSubGroup(base_layer, name=f"Hashed Points Layer {layer_index + 1}")
        map_grid.add_child(hashed_layer)

        for lat, lon in layer_points:
            # Add a small circle around the hashed point
            folium.Circle(
                location=(lat, lon),
                radius=20,  # Medium-sized circle
                color=layer_color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1} Hashed Point"
            ).add_to(hashed_layer)

            # Add a marker on top of the hashed point
            folium.Marker(
                location=(lat, lon),
                icon=folium.Icon(color=layer_color, icon="info-sign"),
                popup=f"Hashed Point ({lat:.5f}, {lon:.5f}) - Layer {layer_index + 1}"
            ).add_to(hashed_layer)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Generate the map with filtering options
grid_map = visualize_grid_with_trajectory(GRID, true_trajectory, hashed_trajectory)

# Save the map
grid_map